In [6]:
import warnings
warnings.filterwarnings("ignore")
import os
import contextlib
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    from manim import *
import autograd.numpy as np


class LectureScene(Scene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.camera.background_color = "#ffffff"
        self.template = TexTemplate()
        self.template.add_to_preamble(r"\usepackage{amsmath}")

class ThreeDLectureScene(ThreeDScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.camera.background_color = "#ffffff"
        self.template = TexTemplate()
        self.template.add_to_preamble(r"\usepackage{amsmath}")
    

class VectorScene(LectureScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ax = Axes(
            x_range=[-7.5, 7.5, 1],
            y_range=[-5, 5, 1],
            x_length=12,
            y_length=8,
            axis_config={"color": GREY},
        )
        
        #axes_labels.set_color(GREY)
        self.add(self.ax)

class PositiveVectorScene(LectureScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ax = Axes(
            x_range=[-2.5, 12.5, 1],
            y_range=[-1, 9, 1],
            x_length=12,
            y_length=8,
            axis_config={"color": GREY},
        )
                #axes_labels.set_color(GREY)
        self.add(self.ax)

class ComparisonVectorScene(LectureScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ax1 = Axes(
            x_range=[-5, 5, 1],
            y_range=[-5, 5, 1],
            x_length=6,
            y_length=6,
            axis_config={"color": GREY},
        )
        self.ax2 = Axes(
            x_range=[-5, 5, 1],
            y_range=[-5, 5, 1],
            x_length=6,
            y_length=6,
            axis_config={"color": GREY},
        )
        axgroup = Group(self.ax1, self.ax2)
        axgroup.arrange_in_grid(buf=2)
        
        #axes_labels.set_color(GREY)
        self.add(axgroup)
from manim_ml.neural_network import NeuralNetwork, Convolutional2DLayer, MaxPooling2DLayer

In [7]:
type(BLACK)

str

In [231]:
%%manim -pql -v WARNING Conv
def make_sequence(numbers, sl=0.5, colors=BLACK, fills=WHITE, text_colors=None, opacity=1.,  annot=True, stride=1, padding=0):
    n = len(numbers)
    squares = []

    if type(colors) is str:
        colors = [colors] * n
    if text_colors is None:
        text_colors = colors
    elif type(text_colors) is str:
        text_colors = [text_colors] * n
    if type(fills) is str:
        fills = [fills] * n

    for i in range(n):
        pos = i * stride * sl * RIGHT
        square = Square(color=colors[i], fill_color=fills[i], side_length=sl,  fill_opacity=opacity).move_to(pos)
        if annot:
            text = Text(str(numbers[i]), color=text_colors[i]).scale(sl).move_to(pos)
            squares.append(VGroup(square, text))
        else:
            squares.append(VGroup(square))
    
    if padding > 0:
        pads = []
        for i in range(padding):
            pos = (n + i) * sl * RIGHT
            pads.append(Text('0', color=GRAY).scale(sl).move_to(pos))
            pos = (-(i + 1)) * sl * RIGHT
            pads.append(Text('0', color=GRAY).scale(sl).move_to(pos))
        return VGroup(*squares), VGroup(*pads)

    return VGroup(*squares)

def make_stack(numbers, sl=0.5, colors=BLACK, fills=WHITE, text_colors=None, opacity=1., annot=True, stride=1):
    n = len(numbers)

    if type(colors) is str:
        colors = [colors] * n
    if text_colors is None:
        text_colors = colors
    elif type(text_colors) is str:
        text_colors = [text_colors] * n
    if type(fills) is str:
        fills = [fills] * n

    seqs = []
    for i in range(n):
        seqs.append(make_sequence(numbers[i], sl, colors[i], fills[i], text_colors[i], opacity, annot, stride).shift(i * sl * DOWN))
    return VGroup(*seqs)

class Conv(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -1 * UP))
        self.add(Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT))
        self.add(Text('Convolution (kernel size: 3)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))


        kernelarr = [-1, 3, -1]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=0.2, fills=RED, colors=RED).move_to(-1 * UP)
        seqarr = np.random.randint(-9, 9, (15,))

        sequence = make_sequence(seqarr, annot=True).move_to(-2 * UP)
        outputarr = [sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]

        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(0 * UP)

        output_window = make_sequence([1], annot=False, opacity=.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, fills=RED, opacity=0.2, colors=RED).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.75 * UP, -1.25 * UP, color=RED).shift(s * RIGHT)
            lab = MathTex(r'\times', color=RED).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=RED)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=RED).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)

        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * 6 * LEFT)
        eq = Text(f'Output[{0 :2}]=({seqarr[0] :2})({kernelarr[0] :2})+({seqarr[1] :2})({kernelarr[1] :2})+({seqarr[2] :2})({kernelarr[2] :2})={outputarr[0] :3}', color=BLACK, font_size=36, font='Roboto Mono').shift(2 * UP)
        self.add(sequence, output, highlight, eq)

                
        output[0][1].set_color(BLACK)
        for i in range(12):
            neweq = Text(f'Output[{i+1 :2}]=({seqarr[0 + i] :2})({kernelarr[0] :2})+({seqarr[1 + i] :2})({kernelarr[1] :2})+({seqarr[2 + i] :2})({kernelarr[2] :2})={outputarr[1 + i] :3}', color=BLACK, font_size=36, font='Roboto Mono').shift(2 * UP)
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), FadeIn(neweq), FadeOut(eq))
            eq = neweq
        self.wait(2)


In [258]:
%%manim -pql -v WARNING Conv

class Conv(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -1 * UP))
        output_label = Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT)
        self.add(output_label)
        #self.add(Text('Convolution (kernel size: 3)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))


        kernelarr = [-1, 3, -1]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=0.2, fills=RED, colors=RED).move_to(-1 * UP)
        seqarr = np.random.randint(-9, 9, (15,))

        sequence = make_sequence(seqarr, annot=True).move_to(-2 * UP)
        outputarr = [sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]

        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(0 * UP)

        output_window = make_sequence([1], annot=False, opacity=.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, fills=RED, opacity=0.2, colors=RED).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.75 * UP, -1.25 * UP, color=RED).shift(s * RIGHT)
            lab = MathTex(r'\times', color=RED).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=RED)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=RED).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)

        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * 6 * LEFT)
        eq = Text(f'Output[{0 :2}]=({seqarr[0] :2})({kernelarr[0] :2})+({seqarr[1] :2})({kernelarr[1] :2})+({seqarr[2] :2})({kernelarr[2] :2})={outputarr[0] :3}', color=BLACK, font_size=36, font='Roboto Mono').shift(2 * UP)
        self.add(sequence, output, highlight, eq)

                
        output[0][1].set_color(BLACK)
        for i in range(12):
            neweq = Text(f'Output[{i+1 :2}]=({seqarr[0 + i] :2})({kernelarr[0] :2})+({seqarr[1 + i] :2})({kernelarr[1] :2})+({seqarr[2 + i] :2})({kernelarr[2] :2})={outputarr[1 + i] :3}', color=BLACK, font_size=36, font='Roboto Mono').shift(2 * UP)
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), FadeIn(neweq), FadeOut(eq), )
            eq = neweq

        kernel_window = VGroup(*lines)
        self.wait(1)
        self.play(FadeOut(kernel_window), FadeOut(output_window), run_time=0.25)
        self.play(kernel.animate.shift(RIGHT * 2), output.animate.shift(1.5 * UP), output_label.animate.shift(1.5 * UP), run_time=0.5, )



        kernelarr = [0, 0, 0]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=0.2, fills=GREEN, colors=GREEN).move_to(-1 * UP)

        kernel_window = make_sequence([1, 1, 1], annot=False, fills=GREEN, opacity=0.2, colors=GREEN).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.75 * UP, -1.25 * UP, color=GREEN).shift(s * RIGHT)
            lab = MathTex(r'+', color=GREEN).scale(0.5).next_to(l, 0.5 * LEFT) 
            lines += [l, lab]
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)
        highlight = VGroup(kernel_window, kernel)
        highlight.shift(0.5 * 6 * LEFT)
        self.add(sequence, output, highlight)

        for i in range(12):
            kernelarr[0] += seqarr[0 + i]
            kernelarr[1] += seqarr[1 + i]
            kernelarr[2] += seqarr[2 + i]
            
            new0 = Text(str(kernelarr[0]), color=GREEN).move_to(kernel[0][1].get_center()).scale(0.5)
            new1 = Text(str(kernelarr[1]), color=GREEN).move_to(kernel[1][1].get_center()).scale(0.5)
            new2 = Text(str(kernelarr[2]), color=GREEN).move_to(kernel[2][1].get_center()).scale(0.5)
            self.play(kernel[0][1].animate.become(new0), kernel[1][1].animate.become(new1), kernel[2][1].animate.become(new2), run_time=0.1)
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), run_time=0.5)
        self.wait(2)



In [315]:
%%manim -pql -v WARNING Conv

from copy import deepcopy
colors = [GOLD, RED, PURPLE]
class Conv(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -1 * UP))
        output_label = Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT)
        self.add(output_label)
        self.add(Text('Convolution (kernel size: 3)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))


        kernelarr = [-1, 3, -1]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=1., colors=GRAY_E, fills=colors).move_to(-1 * UP)
        kernel.z_index=10
        seqarr = np.random.randint(-9, 9, (15,))

        sequence = make_sequence(seqarr, annot=True).move_to(-2 * UP)
        outputarr = [sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]

        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(0 * UP)

        output_window = make_sequence([1], annot=False, opacity=.2, fills=GRAY_E, colors=GRAY_E).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, fills=colors, opacity=0.2, colors=GRAY_E, ).move_to(-2 * UP)

        lines = []
        static_lines = []
        for s, c in zip([-0.5, 0., 0.5], colors):
            l = Line(-1.75 * UP, -1.25 * UP, color=c).shift(s * RIGHT)
            lab = MathTex(r'\times', color=c).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=c)
            lines += [l, lab, l2]

            l = Line(-1.75 * UP + s * RIGHT, -.25 * UP, color=c)#.shift()
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=c)
            static_lines += [l2]

        lines.append(MathTex(r'+', color=RED).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        static_lines = VGroup(*static_lines)
        

        kernel_window = VGroup(*lines)

        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * 6 * LEFT)
        eq = Text(f'Output[{0 :2}]=({seqarr[0] :2})({kernelarr[0] :2})+({seqarr[1] :2})({kernelarr[1] :2})+({seqarr[2] :2})({kernelarr[2] :2})={outputarr[0] :3}', color=BLACK, font_size=36, font='Roboto Mono').shift(2 * UP)
        self.add(sequence, output, highlight)

                
        output[0][1].set_color(BLACK)
        for i in range(12):
            neweq = Text(f'Output[{i+1 :2}]=({seqarr[0 + i] :2})({kernelarr[0] :2})+({seqarr[1 + i] :2})({kernelarr[1] :2})+({seqarr[2 + i] :2})({kernelarr[2] :2})={outputarr[1 + i] :3}', color=BLACK, font_size=36, font='Roboto Mono').shift(2 * UP)
            sl = deepcopy(static_lines).move_to(highlight)
            sl.z_index = -100
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), FadeIn(sl) )
            eq = neweq

        return

        kernel_window = VGroup(*lines)
        self.wait(1)
        self.play(FadeOut(kernel_window), FadeOut(output_window), run_time=0.25)
        self.play(kernel.animate.shift(0.5 * RIGHT), output.animate.shift(.5 * UP), output_label.animate.shift(.5 * UP), sequence.animate.shift(.5 * DOWN), run_time=0.5, )


        seqarr = np.random.randint(-9, 9, (15,))

        sequence = make_sequence(seqarr, annot=True, text_colors=WHITE).move_to(-2 * UP)
        outputarr = np.random.randint(-9, 9, (13,))

        output = make_sequence(outputarr, annot=True).move_to(0 * UP)

        self.play(FadeIn(output), FadeIn(sequence), kernel.animate.set_color(GREEN))
        self.play(Rotate(kernel, 180 * DEGREES))

        output_window = make_sequence([1], annot=False, opacity=.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, fills=RED, opacity=0.2, colors=RED).move_to(-2 * UP)

        output_window = make_sequence([1], annot=False, opacity=.2, fills=GREEN, colors=GREEN).move_to(-2 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, fills=GREEN, opacity=0.2, colors=GREEN).move_to(0 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-.75 * UP, -.25 * UP, color=GREEN).shift(s * RIGHT)
            lab = MathTex(r'\times', color=GREEN).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-1.75 * UP, -1.5 * UP, -1.5 * UP + s * RIGHT, -1.25 * UP + s * RIGHT, color=GREEN)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=GREEN).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        lines.append(output_window)
        kernel_window = VGroup(*lines)
        kernel_window.move_to(kernel)
        self.play(FadeIn(kernel_window), FadeIn(output_window))


        highlight = VGroup(output_window, kernel_window, kernel)
        sequence[-1][1].set_color(BLACK)
        for i in range(13):
            self.play(highlight.animate.shift(LEFT * 0.5), sequence[(14 - i -1)][1].animate.set_color(BLACK) )
            eq = neweq
        self.play(highlight.animate.shift(LEFT * 0.5), sequence[(0)][1].animate.set_color(BLACK) )
        self.wait(2)

In [290]:
%%manim -pql -v WARNING Conv

class Conv(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -1 * UP))
        output_label = Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT)
        self.add(output_label)
        self.add(Text('Convolution (kernel size: 3)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))


        kernelarr = [-1, 3, -1]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=0.2, fills=RED, colors=RED).move_to(-1 * UP)
        seqarr = np.random.randint(-9, 9, (15,))

        sequence = make_sequence(seqarr, annot=True).move_to(-2 * UP)
        outputarr = [sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]

        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(0 * UP)

        output_window = make_sequence([1], annot=False, opacity=.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, fills=RED, opacity=0.2, colors=RED).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.75 * UP, -1.25 * UP, color=RED).shift(s * RIGHT)
            lab = MathTex(r'\times', color=RED).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=RED)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=RED).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)

        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * 6 * LEFT)
        eq = Text(f'Output[{0 :2}]=({seqarr[0] :2})({kernelarr[0] :2})+({seqarr[1] :2})({kernelarr[1] :2})+({seqarr[2] :2})({kernelarr[2] :2})={outputarr[0] :3}', color=BLACK, font_size=36, font='Roboto Mono').shift(2 * UP)
        self.add(sequence, output, highlight)

                
        output[0][1].set_color(BLACK)
        for i in range(12):
            neweq = Text(f'Output[{i+1 :2}]=({seqarr[0 + i] :2})({kernelarr[0] :2})+({seqarr[1 + i] :2})({kernelarr[1] :2})+({seqarr[2 + i] :2})({kernelarr[2] :2})={outputarr[1 + i] :3}', color=BLACK, font_size=36, font='Roboto Mono').shift(2 * UP)
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK) )
            eq = neweq

        kernel_window = VGroup(*lines)
        self.wait(1)
        self.play(FadeOut(kernel_window), FadeOut(output_window), run_time=0.25)
        self.play(kernel.animate.shift(0.5 * RIGHT), output.animate.shift(.5 * UP), output_label.animate.shift(.5 * UP), sequence.animate.shift(.5 * DOWN), run_time=0.5, )


        seqarr = np.random.randint(-9, 9, (15,))

        sequence = make_sequence(seqarr, annot=True, text_colors=WHITE).move_to(-2 * UP)
        outputarr = np.random.randint(-9, 9, (13,))

        output = make_sequence(outputarr, annot=True).move_to(0 * UP)

        self.play(FadeIn(output), FadeIn(sequence), kernel.animate.set_color(GREEN))
        self.play(Rotate(kernel, 180 * DEGREES))

        output_window = make_sequence([1], annot=False, opacity=.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, fills=RED, opacity=0.2, colors=RED).move_to(-2 * UP)

        output_window = make_sequence([1], annot=False, opacity=.2, fills=GREEN, colors=GREEN).move_to(-2 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, fills=GREEN, opacity=0.2, colors=GREEN).move_to(0 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-.75 * UP, -.25 * UP, color=GREEN).shift(s * RIGHT)
            lab = MathTex(r'\times', color=GREEN).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-1.75 * UP, -1.5 * UP, -1.5 * UP + s * RIGHT, -1.25 * UP + s * RIGHT, color=GREEN)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=GREEN).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        lines.append(output_window)
        kernel_window = VGroup(*lines)
        kernel_window.move_to(kernel)
        self.play(FadeIn(kernel_window), FadeIn(output_window))


        highlight = VGroup(output_window, kernel_window, kernel)
        sequence[-1][1].set_color(BLACK)
        for i in range(13):
            self.play(highlight.animate.shift(LEFT * 0.5), sequence[(14 - i -1)][1].animate.set_color(BLACK) )
            eq = neweq
        self.play(highlight.animate.shift(LEFT * 0.5), sequence[(0)][1].animate.set_color(BLACK) )
        self.wait(2)

In [ ]:
%%manim -pql -v WARNING Conv

class Conv(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -1 * UP))
        self.add(Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT))
        self.add(Text('Convolution (kernel size: 5)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))


        kernelarr = [1, -1, 3, -1, 1]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=0.2, fills=RED, colors=RED).move_to(-1 * UP)
        seqarr = np.random.randint(-9, 9, (15,))

        sequence = make_sequence(seqarr, annot=True).move_to(-2 * UP)
        outputarr = [sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]

        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(0 * UP)

        output_window = make_sequence([1], annot=False, opacity=.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1, 1, 1], annot=False, fills=RED, opacity=0.2, colors=RED).move_to(-2 * UP)

        lines = []
        for s in [-1., -0.5, 0., 0.5, 1.]:
            l = Line(-1.75 * UP, -1.25 * UP, color=RED).shift(s * RIGHT)
            lab = MathTex(r'\times', color=RED).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=RED)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=RED).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)

        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * 5 * LEFT)
        self.add(sequence, output, highlight)

                
        output[0][1].set_color(BLACK)
        for i in range(10):
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK))
        self.wait(2)

In [230]:
%%manim -pql -v WARNING Shuffle

class Shuffle(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -1 * UP))
        self.add(Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT))
        self.add(Text('Convolution (size: 3) + Shuffle', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))

        kernelarr = [-1, 3, -1]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=0.2, fills=RED, colors=RED).move_to(-1 * UP)
        seqarr = np.random.randint(-9, 9, (16,))

        sequence = make_sequence(seqarr, annot=True).move_to(-2 * UP)
        outputarr = [sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]

        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(0 * UP)

        output_window = make_sequence([1], annot=False, opacity=0.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False,  opacity=0.2, fills=RED, colors=RED).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.75 * UP, -1.25 * UP, color=RED).shift(s * RIGHT)
            lab = MathTex(r'\times', color=RED).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=RED)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=RED).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)
        
        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * 6.5 * LEFT)
        self.add(sequence, output, highlight)

                
        output[0][1].set_color(BLACK)
        for i in range(13):
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), run_time=0.5)
        n=16
        self.play(output_window.animate.set_opacity(0.), kernel_window.animate.set_opacity(0.), sequence.animate.set_opacity(0.5), run_time=0.5)

        stack_anims = []
        for i, o in enumerate(output):
            if i % 2 == 0:
                stack_anims.append(o.animate.shift(RIGHT * 0.25 + 0.25 * UP))
            else:
                stack_anims.append(o.animate.shift(LEFT * 0.25 + 0.25 * DOWN))
        self.play(*stack_anims)

        

        shift_anims = []
        for i, o in enumerate(output):
            j = i // 2
            s = j - (n / 4)
            shift_anims.append(o.animate.shift(LEFT * s * 0.5 + 0.5 * LEFT))
        self.play(*shift_anims)

        self.wait(3)

In [221]:
%%manim -pql -v WARNING MovingAround

class MovingAround(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -1 * UP))
        self.add(Text('Pooling', color=BLUE, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + 1 * UP))
        self.add(Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + 2 * UP))
        self.add(Text('Conv. output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT))
        self.add(Text('Convolution (size: 3) + Max Pooling (size: 3, stride: 2)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))

        kernelarr = [-1, 3, -1]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=0.2, fills=RED, colors=RED).move_to(-1 * UP)
        seqarr = np.random.randint(-9, 9, (16,))

        sequence = make_sequence(seqarr, annot=True).move_to(-2 * UP)
        outputarr = [sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]

        outputarr2 = maxpool1d(outputarr, 3, stride=2)

        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(0 * UP)
        output2 = make_sequence(outputarr2.astype(int), annot=True, stride=2, text_colors=WHITE).move_to(2 * UP + 0.25 * LEFT)

        output_window = make_sequence([1], annot=False, opacity=0.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, opacity=0.2, fills=RED,  colors=RED).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.75 * UP, -1.25 * UP, color=RED).shift(s * RIGHT)
            lab = MathTex(r'\times', color=RED).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=RED)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=RED).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)

        maxpool_window = make_sequence([1, 1, 1], annot=False, opacity=0.2, colors=BLUE, fills=BLUE).move_to(0 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l2 = CubicBezier(0.25 * UP + s * RIGHT, 0.25 * UP + UP + s * RIGHT, UP, 1.75 * UP, color=BLUE)
            lines += [l2]
        lines.append(MathTex(r'\max', color=BLUE).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(maxpool_window)
        maxpool_window = VGroup(*lines)


        maxpool_output_window = make_sequence([1], annot=False, opacity=0.2, colors=BLUE, fills=BLUE).move_to(2 * UP)
        maxpool_highlight = VGroup(maxpool_output_window, maxpool_window).shift(0.5 * 5.5 * LEFT)
        
        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * 6.5 * LEFT)
        self.add(sequence, output, highlight, output2, )

                
        output[0][1].set_color(BLACK)
        for i in range(13):
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), run_time=0.1)
        n=16

        self.play(FadeIn(maxpool_highlight), highlight.animate.set_opacity(0.), output2[0][1].animate.set_color(BLACK), sequence.animate.set_opacity(0.5))

        for i in range(5):
            self.play(maxpool_highlight.animate.shift(RIGHT * 1.), output2[i + 1][1].animate.set_color(BLACK))

        self.play(maxpool_highlight.animate.set_opacity(0.), output.animate.set_opacity(0.5))
        shift_anims = []
        for i, o in enumerate(output2):
            j = i
            s = j - (4)
            shift_anims.append(o.animate.shift(LEFT * s * 0.5 + 0.5 * LEFT))
        self.play(*shift_anims)
        self.wait(3)
        

In [229]:
%%manim -pql -v WARNING MovingAround

class MovingAround(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -1 * UP))
        self.add(Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT))
        self.add(Text('Convolution (size: 3, stride: 2)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))
        kernelarr = [-1, 3, -1]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=0.2, fills=RED, colors=RED).move_to(-1 * UP)
        seqarr = np.random.randint(-9, 9, (15,))

        sequence = make_sequence(seqarr, annot=True).move_to(-2 * UP)
        outputarr = [sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]

        output = make_sequence(outputarr[::2], annot=True, stride=2, text_colors=WHITE).move_to(0 * UP)

        output_window = make_sequence([1], annot=False, opacity=0.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, opacity=0.2, fills=RED, colors=RED).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.75 * UP, -1.25 * UP, color=RED).shift(s * RIGHT)
            lab = MathTex(r'\times', color=RED).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=RED)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=RED).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)
        
        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * 6 * LEFT)
        self.add(sequence, output, highlight)
                
        output[0][1].set_color(BLACK)
        for i in range(6):
            self.play(highlight.animate.shift(RIGHT * 1), output[i + 1][1].animate.set_color(BLACK))

        self.play(output_window.animate.set_opacity(0.), sequence.animate.set_opacity(0.5), kernel_window.animate.set_opacity(0.))
        shift_anims = []
        for i, o in enumerate(output):
            j = i
            s = j - (4)
            shift_anims.append(o.animate.shift(LEFT * s * 0.5 + 0.5 * LEFT))
        self.play(*shift_anims)
        self.wait(2)


In [222]:
%%manim -pql -v WARNING MovingAround

from scipy.ndimage import correlate


class MovingAround(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -1 * UP))
        self.add(Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT))
        self.add(Text('Convolution (size: 3, padding: 1)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))
        kernelarr = [-1, 3, -1]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=0.2, fills=RED, colors=RED).move_to(-1 * UP)
        seqarr = np.random.randint(-9, 9, (15,))

        sequence, pads = make_sequence(seqarr, annot=True, padding=1)
        sequence.move_to(-2 * UP), pads.move_to(-2 * UP)
        outputarr = convolution1d(seqarr, kernelarr, padding=1).astype(int)

        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(0 * UP)

        output_window = make_sequence([1], annot=False, opacity=0.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, opacity=0.2, fills=RED, colors=RED).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.75 * UP, -1.25 * UP, color=RED).shift(s * RIGHT)
            lab = MathTex(r'\times', color=RED).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=RED)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=RED).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)
        
        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * 7 * LEFT)
        
        self.add(sequence, output, pads)
        self.play(FadeIn(highlight), output[0][1].animate.set_color(BLACK))
                
        
        for i in range(14):
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), run_time=0.5)
        
        self.wait(2)
        

In [233]:
%%manim -pql -v WARNING MovingAround

class MovingAround(LectureScene):

    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT))
        self.add(Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + 2 * UP))
        self.add(Text('Convolution (size: 3, channels: 3, output channels: 1)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))
        n = 16
        
        seqarr = np.random.randint(-9, 9, (3, n,))
        sequence = make_stack(seqarr, annot=True).move_to(-2 * UP)


        kernelarr = np.random.randint(-3, 3, (3, 3,))
        kw = len(kernelarr[0]) // 2
        kernel = make_stack(kernelarr, annot=True, opacity=0.2, colors=[RED, GREEN, BLUE], fills=[RED, GREEN, BLUE]).move_to(0 * UP)
        outputarr = convolution1d(seqarr.T, kernelarr).astype(int) #np.zeros(14).astype(int)#[sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]
        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(1.5 * UP)

        output_window = make_sequence([1], annot=False, opacity=0.2, colors=GRAY, fills=GRAY).move_to(1.5 * UP)
        kernel_window = make_stack(kernelarr, annot=False, opacity=0.2, colors=[RED, GREEN, BLUE], fills=[RED, GREEN, BLUE]).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.25 * UP, -.75 * UP, color=GRAY).shift(s * RIGHT)
            lab = MathTex(r'\text{dot}', color=GRAY).scale(0.35).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.25 * UP + UP + s * RIGHT,  UP + s * RIGHT,  UP, .25 * UP + UP, color=GRAY)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=GRAY).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)
        
        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * (n-3) / 2 * LEFT)
        self.add(sequence, output, highlight)
                
        output[0][1].set_color(BLACK)
        for i in range(n - 3):
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), run_time=0.5)

        self.wait(2)

        #stack_anims = []
        #for i, o in enumerate(output):
        #    if i % 2 == 0:
        #        stack_anims.append(o.animate.shift(RIGHT * 0.25 + 0.25 * UP))
        #    else:
        #        stack_anims.append(o.animate.shift(LEFT * 0.25 + 0.25 * DOWN))
        #self.play(*stack_anims)

        #shift_anims = []
        #for i, o in enumerate(output):
        #    j = i // 2
        #    s = j - (n / 4)
        #    shift_anims.append(o.animate.shift(LEFT * s * 0.5))
        #self.play(*shift_anims)

In [227]:
%%manim -pql -v WARNING MovingAround

class MovingAround(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + 0 * UP))
        self.add(Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + 2 * UP))
        self.add(Text('Convolution (size: 3, channels: 3, output channels: 3)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))
        n = 16
        
        seqarr = np.random.randint(-9, 9, (3, n,))
        sequence = make_stack(seqarr, annot=True).move_to(-2 * UP)


        kernelarr = np.random.randint(-3, 3, (3, 3,))
        kw = len(kernelarr[0]) // 2
        kernel = make_stack(kernelarr, annot=True, opacity=0.2, colors=[RED, GREEN, BLUE], fills=[RED, GREEN, BLUE]).move_to(0 * UP)
        outputarr = convolution1d(seqarr.T, kernelarr).astype(int) #np.zeros(14).astype(int)#[sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]
        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(1.5 * UP)
        output_window = make_sequence([1], annot=False, opacity=0.2, colors=GRAY, fills=GRAY).move_to(1.5 * UP)
        kernel_window = make_stack(kernelarr, annot=False, opacity=0.2, colors=[RED, GREEN, BLUE], fills=[RED, GREEN, BLUE]).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.25 * UP, -.75 * UP, color=GRAY).shift(s * RIGHT)
            lab = MathTex(r'\text{dot}', color=GRAY).scale(0.35).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.25 * UP + UP + s * RIGHT,  UP + s * RIGHT,  UP, .25 * UP + UP, color=GRAY)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=GRAY).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)

        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * (n-3) / 2 * LEFT)
        self.add(sequence, output, highlight)
  
        output[0][1].set_color(BLACK)
        for i in range(n - 3):
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), run_time=0.3)

        self.play(FadeOut(kernel_window),  FadeOut(output_window) , run_time=0.25)
        self.play(kernel.animate.move_to(5.5 * RIGHT), run_time=0.5)
        self.play(kernel.animate.shift(2 * UP), run_time=0.5)


        kernelarr = np.random.randint(-3, 3, (3, 3,))
        kw = len(kernelarr[0]) // 2
        kernel = make_stack(kernelarr, annot=True, opacity=0.2, colors=[RED, GREEN, BLUE], fills=[RED, GREEN, BLUE]).move_to(0 * UP)
        outputarr = convolution1d(seqarr.T, kernelarr).astype(int) #np.zeros(14).astype(int)#[sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]
        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(2 * UP)
        output_window = make_sequence([1], annot=False, opacity=0.2, colors=GRAY, fills=GRAY).move_to(2 * UP)
        kernel_window = make_stack(kernelarr, annot=False, opacity=0.2, colors=[RED, GREEN, BLUE], fills=[RED, GREEN, BLUE]).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.25 * UP, -.75 * UP, color=GRAY).shift(s * RIGHT)
            lab = MathTex(r'\text{dot}', color=GRAY).scale(0.35).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.25 * UP + UP + s * RIGHT,  UP + s * RIGHT,  UP, .25 * UP + UP, color=GRAY)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=GRAY).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)
        
        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * (n-3) / 2 * LEFT)
        self.play(FadeIn(output), FadeIn(highlight), run_time=0.5)

        output[0][1].set_color(BLACK)
        for i in range(n - 3):
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), run_time=0.3)

        self.play(FadeOut(kernel_window),  FadeOut(output_window), run_time=0.25)
        self.play(kernel.animate.move_to(5.5 * RIGHT), run_time=0.5)
        self.play(kernel.animate.shift(2 * DOWN),  run_time=0.5)

        kernelarr = np.random.randint(-3, 3, (3, 3,))
        kw = len(kernelarr[0]) // 2
        kernel = make_stack(kernelarr, annot=True, opacity=0.2, colors=[RED, GREEN, BLUE], fills=[RED, GREEN, BLUE]).move_to(0 * UP)
        outputarr = convolution1d(seqarr.T, kernelarr).astype(int) #np.zeros(14).astype(int)#[sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]
        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(2.5 * UP)
        output_window = make_sequence([1], annot=False, opacity=0.2, colors=GRAY, fills=GRAY).move_to(2.5 * UP)
        kernel_window = make_stack(kernelarr, annot=False, opacity=0.2, colors=[RED, GREEN, BLUE], fills=[RED, GREEN, BLUE]).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.25 * UP, -.75 * UP, color=GRAY).shift(s * RIGHT)
            lab = MathTex(r'\text{dot}', color=GRAY).scale(0.35).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.25 * UP + UP + s * RIGHT,  UP + s * RIGHT,  UP, .25 * UP + UP, color=GRAY)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=GRAY).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)

        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * (n-3) / 2 * LEFT)
        self.play(FadeIn(output), FadeIn(highlight), run_time=0.5)

        output[0][1].set_color(BLACK)
        for i in range(n - 3):
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), run_time=0.3)

        self.play(FadeOut(kernel_window),  FadeOut(output_window), run_time=0.25)
        self.play(kernel.animate.move_to(5.5 * RIGHT), run_time=0.5)
        self.play( sequence.animate.set_opacity(0.5), run_time=0.5)

        self.wait(2)

In [22]:
def convolution1d(x, kernel):
    kernel_width = len(kernel)
    input_size = len(x)

    output_size = input_size - (kernel_width - 1)
    output = np.zeros((output_size,))
    for i in range(output_size):
        for j in range(kernel_width):
            output[i] += x[i + j] * kernel[j]
    return output

In [23]:
def convolution1d(x, kernel):
    kernel_width = len(kernel)
    input_size = len(x)

    output_size = input_size - (kernel_width - 1)
    output = np.zeros((output_size,))
    for i in range(output_size):
        for j in range(kernel_width):
            output[i] += np.dot(x[i + j], kernel[j])
    return output

In [24]:
def convolution1d(x, kernel, padding=0):

    kernel_width = len(kernel)
    input_size = len(x)

    output_size = input_size - (kernel_width - 1) + 2 * padding
    output = np.zeros((output_size,))
    for i in range(output_size):
        for j in range(kernel_width):
            ind = i + j - padding
            if ind >= 0 and ind < input_size:
                output[i] += np.dot(x[ind], kernel[j])
    return output

In [25]:
def convolution1d(x, kernel, padding=0, stride=1):

    kernel_width = len(kernel)
    input_size = len(x)

    output_size = (input_size - (kernel_width - 1) + 2 * padding - 1) // stride + 1
    output = np.zeros((output_size,))
    for i in range(len(output)):
        for j in range(kernel_width):
            ind = i * stride + j - padding
            if ind >= 0 and ind < input_size:
                output[i] += np.dot(x[ind], kernel[j])
    return output

In [26]:
def convolution1d(x, kernel, padding=0, stride=1, dialation=1):

    kernel_width = len(kernel)
    input_size = len(x)

    output_size = (input_size - dialation * (kernel_width - 1) + 2 * padding - 1) // stride + 1
    output = np.zeros((output_size,))
    for i in range(len(output)):
        for j in range(kernel_width):
            ind = i * stride + j * dialation - padding
            if ind >= 0 and ind < input_size:
                output[i] += np.dot(x[ind], kernel[j])
    return output

In [27]:
def convolution1d(x, kernel, bias=0, padding=0, stride=1, dialation=1):

    kernel_width = len(kernel)
    input_size = len(x)

    output_size = (input_size - dialation * (kernel_width - 1) + 2 * padding - 1) // stride + 1
    output = np.zeros((output_size,))
    for i in range(len(output)):
        output[i] = bias
        for j in range(kernel_width):
            ind = i * stride + j * dialation - padding
            if ind >= 0 and ind < input_size:
                output[i] += np.dot(x[ind], kernel[j])
    return output

In [28]:
def maxpool1d(x, window, padding=0, stride=1):
    input_size = len(x)

    output_size = (input_size - (window - 1) + 2 * padding - 1) // stride + 1
    output = np.zeros((output_size,))
    for i in range(len(output)):
        output[i] = -np.infty
        for j in range(window):
            ind = i * stride + j - padding
            if ind >= 0 and ind < input_size:
                output[i] = np.maximum(x[ind], output[i])
    return output

In [1]:

from manim import *

Manim Community v0.17.3

In [4]:
%%manim -pql -v WARNING MovingAround

from manim_ml.neural_network import NeuralNetwork, FeedForwardLayer
class MovingAround(Scene):
    def construct(self):
        nn = NeuralNetwork([
            FeedForwardLayer(num_nodes=3),
            FeedForwardLayer(num_nodes=5),
            FeedForwardLayer(num_nodes=3)
        ])
        self.add(nn)
        # Make the animation
        forward_pass_animation = nn.make_forward_pass_animation()
        # Play the animation
        self.play(forward_pass_animation)

Constructing layers
Current layer: FeedForwardLayer
Current layer: FeedForwardLayer
Current layer: FeedForwardLayer
NeuralNetwork([
    FeedForwardLayer(z_index=3, title_text= , ),
    FeedForwardToFeedForward(input_layer=FeedForwardLayer,output_layer=FeedForwardLayer,)(z_index=2, title_text= , ),
    FeedForwardLayer(z_index=3, title_text= , ),
    FeedForwardToFeedForward(input_layer=FeedForwardLayer,output_layer=FeedForwardLayer,)(z_index=2, title_text= , ),
    FeedForwardLayer(z_index=3, title_text= , ),
])


In [6]:
import manim_ml

In [7]:
manim_ml.ThreeDScene

<module 'manim_ml' from '/Users/gabe/Documents/Courses/CS152-Neural-Networks-Fall-2023.github.io/.venv/lib/python3.9/site-packages/manim_ml/__init__.py'>